In [1]:
import pandas as pd

In [4]:
df = pd.read_pickle('nyc_taxi_prepped.pickle')

In [5]:
df['trip_duration_min']=df['trip_time_in_secs']/60


In [171]:
def create_df(table, column_name): 
    table = pd.DataFrame(table) 
    table.columns = [column_name]
    table = table.reset_index()
    
    return table

In [258]:
driver_no_trips = create_df(df.groupby('medallion')['medallion'].count(), 'trip_count')

In [259]:
driver_total_earnings = create_df(df.groupby('medallion')['fare_amount'].sum(), 'total_earnings')

In [260]:
driver_days_worked =  create_df(df.groupby(['medallion','pickup_day_date'])['pickup_day_date'].count(), 'days_worked')

In [261]:
driver_days_worked = df.drop_duplicates(subset=['medallion', 'pickup_day_date'], keep='first')

In [262]:
driver_days_worked2 = create_df(driver_days_worked.groupby(['medallion'])['pickup_day_date'].count(), "days_worked") 

In [263]:
wkdy_wknd_worked = create_df(driver_days_worked.groupby(['medallion', 'pickup_day_type'])["pickup_day_type"].count(), 'count')

In [264]:
wkdy_wknd_worked2_wknd = wkdy_wknd_worked[wkdy_wknd_worked['pickup_day_type'] == 'Weekend'][['medallion', 'count']]

In [265]:
wkdy_wknd_worked2_wknd.columns = ['medallion','Weekend']

In [266]:
wkdy_wknd_worked2_wkdy = wkdy_wknd_worked[wkdy_wknd_worked['pickup_day_type'] == 'Weekday'][['medallion', 'count']]
wkdy_wknd_worked2_wkdy.columns = ['medallion','Weekday']

In [267]:
driver_table = driver_no_trips.merge(driver_total_earnings).merge(driver_days_worked2).merge(wkdy_wknd_worked2_wknd).merge(wkdy_wknd_worked2_wkdy)

In [268]:
driver_table['earnings_quantile']= pd.qcut(driver_table['total_earnings'], q = 4, labels = False)

In [270]:
driver_table['target_quantile'] = driver_table['earnings_quantile'].apply(lambda x:x +1) 

In [283]:
def create_df(table, name): 
    table = pd.DataFrame(table)
    table.columns = [name]
    table = table.reset_index()
    
    return table
    
    

In [285]:
mean_table = create_df(driver_table.groupby('earnings_quantile')['trip_count'].mean(), 'mean_trip_count')

In [286]:
driver_table.merge(mean_table, how = 'outer')

,medallion,trip_count,total_earnings,days_worked,Weekend,Weekday,earnings_quantile,target_quantile,mean_trip_count
0,00005007A9F30E289E760362F69E4EAD,1333,13739.5,30,8,22,3,4,1359.429790
1,000351EDC735C079246435340A54C7C1,1482,15405.0,30,8,22,3,4,1359.429790
2,0009986BDBAB2F9A125FEF49D0BFCCDD,1427,14409.0,30,8,22,3,4,1359.429790
3,001DFAC01BC0A32F48C3769DD1414778,1205,13160.0,30,8,22,3,4,1359.429790
4,003EEA559FA61800874D4F6805C4A084,1519,15826.5,30,8,22,3,4,1359.429790
...,...,...,...,...,...,...,...,...,...
13127,FFE1AB76511357473BE3236025321493,842,8680.5,30,8,22,0,1,550.172099
13128,FFE25FFFBDE6A6019A6A9051C4E798FA,556,6538.5,30,8,22,0,1,550.172099
13129,FFEC9171E009541071F4CAB49DF113C9,629,6894.5,26,4,22,0,1,550.172099
13130,FFED7CFCB1A658ADEC2DC6A899047060,544,6159.5,26,6,20,0,1,550.172099


In [287]:
mean_table.columns = ['target_quantile', 'mean_trip_count']

In [296]:
driver_table = driver_table.merge(mean_table, how = 'outer')

In [289]:
earnings_table = create_df(driver_table.groupby('earnings_quantile')['total_earnings'].mean(), 'target_earnings')

In [291]:
earnings_table.columns = ['target_quantile', 'target_earnings']

In [293]:
driver_table = driver_table.merge(earnings_table, how = 'outer')

In [297]:
driver_table['add_trip_target'] = driver_table['trip_count'] - driver_table['mean_trip_count']

In [300]:
driver_table['earnings_gap'] = driver_table['target_earnings'] - driver_table['total_earnings']

In [302]:
def find_incentive(x):
    
    fees = 0.1
    profit = 0.5
    
    budget = 0.5*0.1*x
    
    return budget

In [303]:
driver_table['incentive_budget'] = driver_table['earnings_gap'].apply(lambda x:find_incentive(x))

In [304]:
driver_table['incentive_per_trip']  = driver_table['incentive_budget']/driver_table['add_trip_target']